# Dinosaur Name Generation with RNN

In [26]:
import numpy as np
from utils import *
import copy

## 1. Dataset

In [23]:
# Load dataset
data = open('dinos.txt', 'r').read()
# data, type=str
# Aachenosaurus
# Aardonyx
# Abdallahsaurus
# Abelisaurus
# ...
# Zuoyunlong
# Zupaysaurus
# Zuul
data = data.lower()
chars = list(set(data)) # ['r', 'n', 'l', 'h', 'm', 'b', 'a', 'o', 's', 'k', 'w', 'x', 'd', 'c', 'g', 'u', 'q', 'j', 'e', 'p', 't', 'f', 'y', 'z', 'i', '\n', 'v']
data_size, vocab_size = len(data), len(chars)
print('There are %d total characters and %d unique characters in your data.' % (data_size, vocab_size))

There are 19909 total characters and 27 unique characters in your data.


In [3]:
# Tạo hai dict char_to_ix và ix_to_char
char_to_ix = { ch:i for i,ch in enumerate(chars) } 
# {'r': 0, 'n': 1, 'l': 2, 'h': 3, 'm': 4, 'b': 5, 'a': 6, 'o': 7, 's': 8, 'k': 9, 'w': 10, 'x': 11, 'd': 12, 'c': 13, 'g': 14, 'u': 15, 'q': 16, 'j': 17, 'e': 18, 'p': 19, 't': 20, 'f': 21, 'y': 22, 'z': 23, 'i': 24, '\n': 25, 'v': 26}

ix_to_char = { i:ch for i,ch in enumerate(chars) }
# {0: 'r', 1: 'n', 2: 'l', 3: 'h', 4: 'm', 5: 'b', 6: 'a', 7: 'o', 8: 's', 9: 'k', 10: 'w', 11: 'x', 12: 'd', 13: 'c', 14: 'g', 15: 'u', 16: 'q', 17: 'j', 18: 'e', 19: 'p', 20: 't', 21: 'f', 22: 'y', 23: 'z', 24: 'i', 25: '\n', 26: 'v'}

Các hàm sau đã có sẵn trong utils.py
```python
def rnn_forward(X, Y, a_prev, parameters):
    """ Performs the forward propagation through the RNN and computes the cross-entropy loss.
    It returns the loss' value as well as a "cache" storing values to be used in backpropagation."""
    ....
    return loss, cache
    
def rnn_backward(X, Y, parameters, cache):
    """ Performs the backward propagation through time to compute the gradients of the loss with respect
    to the parameters. It returns also all the hidden states."""
    ...
    return gradients, a

def update_parameters(parameters, gradients, learning_rate):
    """ Updates parameters using the Gradient Descent Update Rule."""
    ...
    return parameters
```

In [47]:
def clip(gradients, maxValue): # Hàm phụ để tối ưu tối hơn
    gradients = copy.deepcopy(gradients)
    
    dWaa, dWax, dWya, db, dby = gradients['dWaa'], gradients['dWax'], gradients['dWya'], gradients['db'], gradients['dby']
   
    # Clip to mitigate exploding gradients, loop over [dWax, dWaa, dWya, db, dby]. (≈2 lines)
    for gradient in gradients:
        np.clip(gradients[gradient], -maxValue, maxValue, out = gradients[gradient])
    
    gradients = {"dWaa": dWaa, "dWax": dWax, "dWya": dWya, "db": db, "dby": dby}
    
    return gradients


def optimize(X, Y, a_prev, parameters, learning_rate = 0.01):
    # Forward propagate through time (≈1 line)
    loss, cache = rnn_forward(X, Y, a_prev, parameters)
    
    # Backpropagate through time (≈1 line)
    gradients, a = rnn_backward(X, Y, parameters, cache)
    
    # Clip your gradients between -5 (min) and 5 (max) (≈1 line)
    gradients = clip(gradients, 5)
    
    # Update parameters (≈1 line)
    parameters = update_parameters(parameters, gradients, learning_rate)
    
    return loss, gradients, a[len(X)-1]


def sample(parameters, char_to_ix, seed):
    """
    Sample a sequence of characters according to a sequence of probability distributions output of the RNN

    Arguments:
    parameters -- Python dictionary containing the parameters Waa, Wax, Wya, by, and b. 
    char_to_ix -- Python dictionary mapping each character to an index.
    seed -- Used for grading purposes. Do not worry about it.

    Returns:
    indices -- A list of length n containing the indices of the sampled characters.
    """

    # Retrieve parameters and relevant shapes from "parameters" dictionary
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    vocab_size = by.shape[0]
    n_a = Waa.shape[1]

    # Step 1: Create the a zero vector x that can be used as the one-hot vector 
    # Representing the first character (initializing the sequence generation). (≈1 line)
    x = np.zeros((vocab_size,1))
    # Step 1': Initialize a_prev as zeros (≈1 line)
    a_prev = np.zeros((n_a ,1))

    # Create an empty list of indices. This is the list which will contain the list of indices of the characters to generate (≈1 line)
    indices = []

    # idx is the index of the one-hot vector x that is set to 1
    # All other positions in x are zero.
    # Initialize idx to -1
    idx = -1

    # Loop over time-steps t. At each time-step:
    # Sample a character from a probability distribution 
    # And append its index (`idx`) to the list "indices". 
    # You'll stop if you reach 50 characters 
    # (which should be very unlikely with a well-trained model).
    # Setting the maximum number of characters helps with debugging and prevents infinite loops. 
    counter = 0
    newline_character = char_to_ix['\n']
    
    while (idx != newline_character and counter != 50):

        # Step 2: Forward propagate x using the equations (1), (2) and (3)
        a = np.tanh(np.dot(Wax,x) + np.dot(Waa,a_prev) + b)
        z = np.dot(Wya,a) + by
        y = softmax(z)

        # For grading purposes
        np.random.seed(counter + seed) 

        # Step 3: Sample the index of a character within the vocabulary from the probability distribution y
        # (see additional hints above)
        idx = np.random.choice(range(len(y)), p = np.squeeze(y))

        # Append the index to "indices"
        indices.append(idx)

        # Step 4: Overwrite the input x with one that corresponds to the sampled index `idx`.
        # (see additional hints above)
        x = np.zeros((vocab_size,1))
        x[idx] = 1

        # Update "a_prev" to be "a"
        a_prev = a

        # for grading purposes
        seed += 1

        counter +=1

    if (counter == 50):
        indices.append(char_to_ix['\n'])
    
    return indices

## 2. Model

<img src="images/rnn.png" height=350/>
<img src="images/dinos3.png" height=350/>

In [48]:
def model(data_x, ix_to_char, char_to_ix, num_iterations = 35000, n_a = 50, dino_names = 7, vocab_size = 27, verbose = False):    
    # Retrieve n_x and n_y from vocab_size
    n_x, n_y = vocab_size, vocab_size # n_x=n_y=27
    
    # Initialize parameters
    parameters = initialize_parameters(n_a, n_x, n_y)
    # parameters={"Wax": Wax, "Waa": Waa, "Wya": Wya, "b": b,"by": by}
    # Wax shape=(n_a, n_x)=(50, 27), Waa shape=(n_a, n_a)=(50, 50), W_ya shape=(n_y, n_a)=(27, 50), b shape=(n_a, 1)=(50, 1), b_y shape=(n_y, 1)=(27, 1).

    # Initialize loss (this is required because we want to smooth our loss)
    loss = get_initial_loss(vocab_size, dino_names)
    
    # Build list of all dinosaur names (training examples).
    examples = [x.strip() for x in data_x] # len=1536, =['aachenosaurus', 'aardonyx', 'abdallahsaurus', 'abelisaurus', ...]
    
    # Shuffle list of all dinosaur names
    np.random.seed(0)
    np.random.shuffle(examples)
    # examples=['turiasaurus', 'pandoravenator', 'ilokelesia', ...]
    
    # Initialize the hidden state of your LSTM
    a_prev = np.zeros((n_a, 1))
    
    # for grading purposes
    last_dino_name = "abc"
    
    # Optimization loop
    for j in range(num_iterations): # Trong một vòng lặp
        # 1. Chọn random một name từ dataset để train bằng cách Lấy index của current example, lấy name tương ứng, rồi chuyển name đó thành list of indices.

        # Set the index `idx` (see instructions above) 
        idx = j%len(examples) # e.g.: j=0, idx=0
        
        # Set the input X (see instructions above)
        single_example_chars = examples[idx] # single_example_chars=turiasaurus
        single_example_ix = [char_to_ix[c] for c in single_example_chars] # single_example_ix=[20, 15, 0, 24, 6, 8, 6, 15, 0, 15, 8]

        # 2. Khởi tạo x^{<1>}=None. Tính training example X bằng cách ghép x^{<1>} với single_example_ix. Tính label Y là X dịch forward đi một và thêm index của `\n`.
        # Ta cần X=None ở đầu để sau đó Y dài bằng X

        # if X[t] == None, we just have x[t]=0. This is used to set the input for the first timestep to the zero vector. 
        X = [None] + single_example_ix # X=[None, 20, 15, 0, 24, 6, 8, 6, 15, 0, 15, 8], len=12
        
        # Set the labels Y (see instructions above)
        # The goal is to train the RNN to predict the next letter in the name
        # So the labels are the list of characters that are one time-step ahead of the characters in the input X
        Y = X[1:] # Y=[20, 15, 0, 24, 6, 8, 6, 15, 0, 15, 8], len=11
        # The RNN should predict a newline at the last letter, so add ix_newline to the end of the labels
        ix_newline = [char_to_ix["\n"]]
        Y = Y + ix_newline # Y=[20, 15, 0, 24, 6, 8, 6, 15, 0, 15, 8, 25], len=12

        # Trong hàm optimize
        # 3. Truyền X và Y vào RNN. Trong hàm rnn_forward
        # - Khởi tạo a0=a_prev=0, có shape (n_a, 1). Đặt a[-1]=a0
        # - Khởi tạo các dicts x, a, y_hat = {}, {}, {}
        # - Lần lượt lặp qua các time steps t. Tại một time step t:
        #     - Tính x[t] là một one-hot vector, có shape (n_x, 1), có giá trị $1$ tại vị trí Xt.
        #       VD tại t=1, X[t]=20, x[t]=[[0], [0], ..., [0], [1], [0]..., [0]], có 1 tại vị trí 20
        #     - Truyền a[t-1] (tại t=0 là a0) và x[t] vào một RNN cell, từ đó tính a[t] và y_hat[t].
        #     y_hat[t] có shape (n_y, 1)=(27, 1), là một probability vector, 
        #     giá trị tại vị trí i thể hiện XS character có index i sẽ là next character.
        #     - Tính loss là negative log của giá trị y_hat[t] tại vị trí Y[t]:      
        #         loss = -log(y_hat[t][Y[t]]).
        # Sau khi lặp xong, dict x có len 12, dict a có len 13, dict y_hat có len 12

        # Perform one optimization step: Forward-prop -> Backward-prop -> Clip -> Update parameters
        # Choose a learning rate of 0.01
        curr_loss, gradients, a_prev = optimize(X, Y, a_prev, parameters, learning_rate = 0.01)
        # Lấy ra loss và a_prev chính là a[t] cuối
        
        
        # debug statements to aid in correctly forming X, Y
        if verbose and j in [0, len(examples) -1, len(examples)]:
            print("j = " , j, "idx = ", idx,) 
        if verbose and j in [0]:
            #print("single_example =", single_example)
            print("single_example_chars", single_example_chars)
            print("single_example_ix", single_example_ix)
            print(" X = ", X, "\n", "Y =       ", Y, "\n")
        
        # Use a latency trick to keep the loss smooth. It happens here to accelerate the training.
        loss = smooth(loss, curr_loss)

        # d. Generate text (có thể gen sau mỗi n iterations). Lúc này model đã đc trained và đc fixed.
        # Every 1000 Iteration, generate "n" characters thanks to sample() to check if the model is learning properly
        if j % 1000 == 0:
            print('Iteration: %d, Loss: %f' % (j, loss) + '\n')
            
            # Sample `dino_names` lần để có `dino_names` generated names. Trong một lần sample:
            # The number of dinosaur names to print
            seed = 0
            for name in range(dino_names):
                # Trong hàm sample:
                # - Khởi tạo x=0, có shape (n_x, 1) và a_prev=0, có shape (n_a, 1).
                # - Dùng while để lặp khi chưa gen ra kí tự "\n"
                # - Chạy một forward propagation để tính a^{<1>} và \hat{y}^{<1>}.
                # - Nếu chọn character có XS cao nhất, model sẽ luôn cho ra cùng KQ cho một starting letter.
                #   Để có nhiều KQ hơn, ta chọn next letter ngẫu nhiên dựa trên XS.
                #   Tạo idx là id của letter đc chọn
                # - Cập nhật x^{<t>} thành x^{<t+1>} bằng cách tạo x là one-hot vector có vị trí 1 tại idx.
                # - Truyền $x^{<t+1>}$ vào step 1 để lặp lại process tới khi có character `\n`.
                # - Output của hàm `sample` là list các idx đã tính, sẽ đc chuyển thành name tương ứng.
                
                # Sample indices and print them
                sampled_indices = sample(parameters, char_to_ix, seed)
                last_dino_name = get_sample(sampled_indices, ix_to_char)
                print(last_dino_name.replace('\n', ''))
                
                seed += 1  # To get the same result (for grading purposes), increment the seed by one. 
      
            print('\n')
        
    return parameters, last_dino_name

In [51]:
parameters, last_name = model(data.split("\n"), ix_to_char, char_to_ix, num_iterations = 11001, verbose = True)

j =  0 idx =  0
single_example_chars turiasaurus
single_example_ix [20, 15, 0, 24, 6, 8, 6, 15, 0, 15, 8]
 X =  [None, 20, 15, 0, 24, 6, 8, 6, 15, 0, 15, 8] 
 Y =        [20, 15, 0, 24, 6, 8, 6, 15, 0, 15, 8, 25] 

Iteration: 0, Loss: 23.087337

Gxviztmcajub
Xgalrmtyolnczxzoad
Xviztmcajub
Galrmtyolnczxzoad
Viztmcajub
Alrmtyolnczxzoad
Iztmcajub


Iteration: 1000, Loss: 28.769617

Koitiqhonusayhus
Osaaraetannopatamoyoll
Oitiqhonusayhus
Kanraetanrseus

Bnraetanrseus
Itenolusayhus


j =  1535 idx =  1535
j =  1536 idx =  0
Iteration: 2000, Loss: 27.868678

Ksitpurus
Osasrus
Ottesaurus
Karraeunmhopapnos

Anrausanrustplasiurothostmripkhonagopsaurus
Ytersaurus


Iteration: 3000, Loss: 26.795501

Saypichongga
Asadragtanlops
Otipclos
Sanraucarns

Mhnaceransus
Zichongda


Iteration: 4000, Loss: 25.849603

Sayceongosaurus
Ablaratensmugoenos
A
Sanraetangdilimos
Iteoniosaulus
Mmrajianhusteras
Yperosaurus


Iteration: 5000, Loss: 25.277315

Kuzytersaurus
Scharateran
S
Wannateran
Itterosaurus
Blnocen

### Notes
* Ở đây tại mỗi vòng lặp training ta chỉ lấy ra một single name thay vì một batch, nên một sample X sẽ có shape $(n_x, T_x)$. Ở đây $n_x=27$ còn $T_x$ thay đổi theo sample đc chọn.
* Khi implement thực tế, ta đã tạo X là một list các indices có len $T_x$ và tạo x là một dict có len $T_x$, mỗi element x[t] có shape $(n_x, 1)$ là one-hot vector, có giá trị $1$ tại index X[t].